In [1]:
import scanpy as sc
import pandas as pd
import os 
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl

import gc
sc.settings.n_jobs = 30
sc._settings.settings._vector_friendly=True

/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-p

In [2]:
color_annotation = pd.read_csv('/home/workspace/IHA_Figure_Revision/Color_hex_codes/Cell_Type_Color_Annotation_and_Order.csv')
color_palette = {} 



In [3]:
adata=sc.read_h5ad("03_UMAP_Subsamples/adata_processed.h5ad")

In [4]:
adata.X = None
adata.raw = None
adata.layers.clear
gc.collect()

41

In [ ]:
visit_list = ['Flu Year 1 Day 0','Flu Year 1 Day 90','Flu Year 2 Day 0']


cmv_celltype = ['Adaptive NK cell', 
                'KLRF1- GZMB+ CD27- memory CD4 T cell',
                'KLRF1- GZMB+ CD27- EM CD8 T cell',
                'KLRF1+ GZMB+ CD27- EM CD8 T cell', 
                'KLRF1- effector Vd1 gdT', 
                'KLRF1+ effector Vd1 gdT']

In [7]:
for cell in cmv_celltype:
    match = color_annotation[color_annotation['label'] == cell]
    if not match.empty:
        color_palette[cell] = match.iloc[0]['color']
color_palette["Other"] = "#a9a9a9"


In [8]:
colors = []

for ct in adata.obs['AIFI_L3']:
    if ct in cmv_celltype:
        colors.append(ct)
    else:
        colors.append('Other')

adata.obs['cell_type_highlight'] = colors

In [9]:
subject_adata = adata[adata.obs['subject.subjectGuid'] == 'BR1049'].copy()

In [11]:
mpl.rcParams.update({
    'font.size': 6,
    'axes.linewidth': 0.53
})
nrows = 1
ncols = 3
fig, axes = plt.subplots(
    nrows, ncols,
    figsize=(60 / 25.4, 18 / 25.4),
    gridspec_kw={'wspace': 0, 'hspace': 0}
)

plot_idx = 0
for visit in visit_list:
    br1_sub = subject_adata[subject_adata.obs["sample.visitName"] == visit].copy()
    ax = axes.flat[plot_idx]
    sc.pl.umap(
        br1_sub,
        color="cell_type_highlight",
        palette=color_palette,
        ax=ax, title='', size=1,
        show=False
    )
    if ax.legend_:
        ax.legend_.remove()
    plot_idx += 1

for ax in axes.flat[plot_idx:]:
    ax.set_visible(False)

for i, ax in enumerate(axes.flat):
    row = i // ncols
    col = i % ncols
    if row != nrows - 1:
        ax.set_xticks([])
        ax.set_xticklabels([])
        ax.set_xlabel('')
    if col != 0:
        ax.set_yticks([])
        ax.set_yticklabels([])
        ax.set_ylabel('')

plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig("FigureB-1.pdf", format="pdf", bbox_inches="tight", dpi=1000)
plt.close()

In [14]:
mpl.rcParams.update({
    'font.size': 6,
    'axes.linewidth': 0.53
})
cmv_values = ['Negative', 'Positive']
nrows = 2
ncols = 2
fig, axes = plt.subplots(
    nrows, ncols,
    figsize=(60/25.4, 60/25.4),
    gridspec_kw={'wspace': 0, 'hspace': 0}
)

plot_idx = 0
n_cells = 30000 

for cohort in ["BR1","BR2"]:
    adata_cohort = adata[adata.obs["cohort.cohortGuid"] == cohort].to_memory()
    for cmv in cmv_values:
        adata_cmv = adata_cohort[adata_cohort.obs["subject.cmv"] == cmv].copy()
        br1_sub = adata_cmv.copy()
        if plot_idx >= nrows * ncols or br1_sub.n_obs == 0:
            continue
        if br1_sub.n_obs > n_cells:
            sc.pp.subsample(br1_sub, n_obs=n_cells, random_state=0)
        ax = axes.flat[plot_idx]
        sc.pl.umap(
            br1_sub,
            color="cell_type_highlight",
            palette=color_palette,
            ax=ax, title='', size=0.15,
            show=False,
        )
        if ax.legend_:
            ax.legend_.remove()
        plot_idx += 1

for i, ax in enumerate(axes.flat):
    row = i // ncols
    col = i % ncols
    if row != nrows - 1:
        ax.set_xticks([])
        ax.set_xticklabels([])
        ax.set_xlabel('')
    if col != 0:
        ax.set_yticks([])
        ax.set_yticklabels([])
        ax.set_ylabel('')

plt.savefig("FigureB-2.pdf", format="pdf", bbox_inches="tight", dpi=1000)
plt.close()